<a href="https://colab.research.google.com/github/antonkozhukhov/breeds/blob/main/dog_breeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
print("Yes, GPU is used" if tf.config.list_physical_devices("GPU") else "No, GPU is not used")
import pandas as pd
import numpy as np
import io, base64

#finding unique labels of breeds
labels = pd.read_csv('drive/MyDrive/dog-vision/labels.csv')
unique_labels = np.unique(labels['breed'])

IMG_SIZE = 224

def process_image(path):
  """
  create image tensor from image
  
  """
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

BATCH_SIZE = 32

def create_data_batches(X, y=None, batch_size=BATCH_SIZE):
  """
  create data batches from image
  
  """
  print("creating test data batches...")
  data = tf.data.Dataset.from_tensors((tf.constant(X)))
  data_batch = data.map(process_image).batch(BATCH_SIZE)
  return data_batch
  
def load_model(model_path):
  """
  loads the model from model path
  
  """
  print("loading model...")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={"KerasLayer":hub.KerasLayer})
  return model


path = 'drive/MyDrive/dog-vision/model/20230422-081712-dogs_5steps.h5'
load_model = load_model(path)

def create_list_of_breeds(b64_string):
  """
  creates the list of 3 most probable breed predictions.
  converts b64_string to jpg image and then using model loaded finds 
  all breed predictions
  
  """
  temporary_image = 'drive/MyDrive/dog-vision/current_image.jpg'
  with open(temporary_image, "wb") as fh:
    fh.write(base64.decodebytes(b64_string))
  val_data = create_data_batches(temporary_image)
  predictions = load_model.predict(val_data, verbose=1)
  pred_n = unique_labels[np.argsort(predictions[0])][-3:][::-1]
  return pred_n.tolist()

# Creating local sever on 5000 Port  using Flask

!pip install -U flask-cors
from flask import *
from flask_cors import CORS, cross_origin

app = Flask(__name__)
cors = CORS(app)
cors = CORS(app, resource={
    r"/*":{
        "origins":"*"
    }
})
app.config['CORS_HEADERS'] = 'Content-Type'
@app.route('/', methods=['POST'])
@cross_origin(allow_headers=['Content-Type'])
def home():
  resp = app.make_default_options_response()
  h = resp.headers
  content_length = int(request.headers['Content-Length'])
  response = app.response_class(
        response=json.dumps(create_list_of_breeds(request.data)),
        status=200,
        mimetype='application/json'
    )
  return response
import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()

# Creating unique subdomain using localtunnel

!npm install -g localtunnel
!lt -p 5000 --subdomain breeds1988 
!nohup lt --port 5000 --subdomain breeds1988 &

Yes, GPU is used
loading model...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.777s
your url is: https://breeds1988.loca.lt
^C
nohup: appending output to 'nohup.out'
